In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import tensorflow as tf

import matplotlib.pyplot as plt
import pickle
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Directories
trainDir="../input/traffic-signs-preprocessed/train.pickle"
validDir="/kaggle/input/traffic-signs-preprocessed/valid.pickle"
testDir="/kaggle/input/traffic-signs-preprocessed/test.pickle"

In [ ]:
# Reading Pickle Files
trainHand=open(trainDir,'rb')
trainHand=pickle.load(trainHand)
#print(trainHand)
testHand=open(testDir,'rb')
testHand=pickle.load(testHand)
validHand=open(validDir,'rb')
validHand=pickle.load(validHand)


In [1]:
# extracting labels and images from pickled data
train_images, train_labels= trainHand['features'],trainHand['labels'] #imagesize(37499,32,32,3) label_size=(37499,)
test_images, test_labels= testHand['features'],testHand['labels'] 
valid_images, valid_labels= validHand['features'],validHand['labels'] 
class_types=len(set(train_labels))
print(len(train_images))
#plt.imshow(images[3000])
#plt.show()

NameError: name 'trainHand' is not defined

In [ ]:
# Normalizing Images
train_images=train_images/255
test_images=test_images/255
valid_images=valid_images/255

In [ ]:
# Not used in the provided code
# Extra feature for preprocessing images
import cv2
'''def preProc(images):
    grayScaled=[cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)/255 for image in images]
    
    return grayScaled

train_images=preProc(train_images)
test_images=preProc(test_images)
valid_images=preProc(valid_images)'''

In [ ]:
# Displays Image
plt.imshow(train_images[7000])
plt.show()
#print(len(train_images[7000]))

In [ ]:
# TensorFlow Sequential Model
model=tf.keras.Sequential([
    tf.keras.layers.Conv2D(64,(3,3),activation='relu',input_shape=(32,32,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(256,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(class_types,activation='softmax')
])
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.summary()


In [ ]:
# Creates Batches of Input Data
def batchData(features,labels,BATCH_SIZE):
    X_items=[]
    tempX=[]
    Y_items=[]
    tempY=[]
    for items in features:
        tempX+=[items]
        if len(tempX)==BATCH_SIZE:
            X_items+=tempX
            tempX=[]
    for items in labels:
        tempY+=[items]
        if len(tempY)==BATCH_SIZE:    
            Y_items+=tempY
            tempY=[]
    return X_items,Y_items


In [ ]:
# Creating Batches using batchData Function
trainData, trainLabels = batchData(train_images,train_labels,64)
validData, validLabels = batchData(valid_images,valid_labels,64)

In [ ]:
# Defining Callback object
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs=[]):
        if logs.get('val_loss') < 0.3:
            print('Validation Accuracy Achieved')
            self.model.stop_training=True
callback=myCallback()

In [ ]:
# Fitting on Training Data
model.reset_states()
model.reset_metrics()
history=model.fit(np.array(trainData),np.array(trainLabels),epochs=10,steps_per_epoch=37499//64,
                  validation_data=(np.array(validData),np.array(validLabels)),callbacks=[callback])

In [ ]:
# Displays Validation and Training Loss to manually observe overfitting
plt.plot(np.arange(1,11),history.history['val_loss'],label='Validation Loss')
plt.plot(np.arange(1,11),history.history['loss'],label='Training Loss')
plt.legend()

In [ ]:
# Displays Test Image
plt.imshow(test_images[30])
plt.show()

In [ ]:
# predicts the image label
print(np.argmax(model.predict(test_images)[30]))
